In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Jan  8 09:57:43 2019

@author: albhsu
"""

#####This is a on-job self-development project. Only the sample of my code is provided#####

import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import auc, roc_curve, classification_report
from sklearn import metrics

# Import Dataset

In [ ]:
data = pd.read_csv('PG_test.csv')
data.head()
data.info()

# Check if any missing data
data.isnull().values.any()

# Machine Learning

In [ ]:
#Split datset into training and testing set
data['is_train'] = np.random.uniform(0, 1, len(data)) <= .75
train, test = data[data['is_train']==True], data[data['is_train']==False]
features = data.columns[6:67]

In [ ]:
#Run Logistic Regression model
y_train = pd.factorize(train['PGDEF'])[0]
clf = LogisticRegression(random_state=0, solver='lbfgs',multi_class='ovr')
clf.fit(train[features], y_train)
y_pred = clf.predict(test[features])

# Model Perfomance

In [ ]:
#Classifier score and Confusion Matrix
clf.score(test[features], test['PGDEF'])
cm = metrics.confusion_matrix(test['PGDEF'], y_pred)
print(cm)

In [ ]:
#Feature importance
importance = pd.DataFrame(index = features, data = clf.feature_importances_, columns = ['importance'])
importance.sort_values(by = 'importance', ascending = True, inplace = True)

In [ ]:
#Plot feature importance
ax = importance[-5:].plot.barh()
y_pos = np.arange(len(features))
plt.barh(y_pos, clf.coef_.ravel())
plt.show()

In [ ]:
#Another way to get Confusion matrix
pd.crosstab(test['y'], y_pred)

#Classification report
target_names = ['0', '1']
y_pred = clf.predict(test[features])
y_actual = test['PGDEF']
print(classification_report(y_actual, y_pred, target_names=target_names))
fpr, tpr, thresholds = metrics.roc_curve(y_pred, y_actual, pos_label=2)